In [2]:
import pandas as pd
import numpy as np
import torch
import os
import sys
import numpy as np
from PIL import Image

# functions

In [1]:
# this is a function to get the coordinates of the patches in the WSIs, according to the tile_selection.tsv file
def get_coordinates(tile_selection_path,patches_size,left_proportion,top_proportion):
    df = pd.read_table(tile_selection_path)
    # select the patches with Keep = 1 and in the lower right side of the WSI based on the proportion
    df = df.loc[(df["Keep"] == 1) & (df["Column"]>=int(max(df["Column"])*left_proportion))& (df["Row"]>=int(max(df["Row"])*top_proportion))]
    # group the rows by index to form a list of tuples
    tuples = [(x, y) for x, y in zip(df['Row']*patches_size, df['Column']*patches_size)]
    patches_num = len(tuples)
    return tuples , patches_num
    

# testing area

In [ ]:
#check if all the slides are processed to patches, and find the number of patches is less than 10000
directory_path = '/home/exon_storage1/aml_slide/patches/'
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        number_of_patches = len(os.listdir(patches_path))
        if number_of_patches <= 10000:
            print( file+" : "+str(len(os.listdir(patches_path))))
    else:
        print(file+" is not processed yet")

In [ ]:
#this script is used to normalize the patches and save them in a new folder, with a blurry filter
sys.path.insert(1, '/home/weber50432/AML_image_processing/HEnorm_python/')
from normalizeStaining import normalizeStaining , is_blurry
directory_path = '/home/weber50432/AML_image_processing/PyHIST/output/'
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        if os.path.exists(directory_path+file+"/rightside_patch_norm"):
            print(file + " rightside_patch_norm is already created !")
            continue
        else:
            os.mkdir(directory_path+file+"/rightside_patch_norm/")
        if os.path.exists(directory_path+file+"/rightside_patch_blurry"):
            print(file + " rightside_patch_blurry is already created !")
        else:
            os.mkdir(directory_path+file+"/rightside_patch_blurry/")
        patches_path_blurry = directory_path+file+"/rightside_patch_blurry/"
        patches_path_norm = directory_path+file+"/rightside_patch_norm/"
        for img in os.listdir(patches_path):
            if img.endswith(".png"):
                img_num = str(img)
                img_path = patches_path+img
                img = np.array(Image.open(img_path))
                Inorm, H, E = normalizeStaining(img = img,
                      saveFile = None,
                      Io = 240,
                      alpha = 1,
                      beta = 0.15)
                if is_blurry(Inorm):
                    Image.fromarray(Inorm).save(patches_path_blurry+img_num)
                    # os.remove(img_path)
                    print(img_num+" is blurry !")
                else:
                    Image.fromarray(Inorm).save(patches_path_norm+img_num)
                    print(img_num+" is saved !")

In [8]:
patches_size = 512
list, num = get_coordinates('/home/weber50432/AML_image_processing/PyHIST/output/A13/tile_selection.tsv',patches_size,0.5,0)
print(list[:5])
print(num)
grid = []
grid.append(list)
output = {
    "slides": ["my/full/path/slide01.svs", "my/full/path/slide02.svs"],
    "grid": grid
    ,
    "targets": [0, 1],
    "mult": patches_size/224,
    "level": 0,
}

torch.save(output, "./data.pt")


[(5120, 121344), (5120, 121856), (5120, 161280), (5632, 117760), (5632, 118272)]
10982


In [9]:
lib = torch.load("./data.pt")
grid = []
slideIDX = []
for i,g in enumerate(lib['grid']):
  grid.extend(g)
  slideIDX.extend([i]*len(g))

print('Number of tiles: {}'.format(len(grid)))

Number of tiles: 10982
